In [19]:
# imports
import spacy
from spacy.matcher import PhraseMatcher
# load default skills data base
from skillNer.general_params import SKILL_DB
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor
from IPython.display import display
import json

In [20]:
# init params of skill extractor
nlp = spacy.load("en_core_web_lg")
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [21]:
''' < reading data > '''
with open('data\jobs_crawl_filter_jobsuche_en.csv', encoding="utf8") as f:
    contents = f.readlines()

In [22]:
JobTitle = list()
JobID = list()

In [23]:
''' < Extracting Title of the Job >'''
for jobs in contents:
    for jobTitle in jobs:
        index = jobs.index(jobTitle)
        if jobTitle.isdecimal() and (jobTitle+jobs[index+1]==jobTitle+'|'):
            jobTitle = jobs.split('|')[2]
            jobId = jobs.split('|')[0]
            if jobTitle not in JobTitle:
                JobTitle.append(jobTitle.lower())
            if jobId not in JobID:
                JobID.append(jobId)
            break

In [24]:
''' < Extracting Job Descriptions of the Job >'''
JobDescription=list()

description = ''
for jobs in contents:
    for jobTitle in jobs:
        index = jobs.index(jobTitle)
        if jobTitle.isdecimal() and (jobTitle+jobs[index+1]==jobTitle+'|'):
            if len(description)>0 and description not in JobDescription:
                # description = description.replace("\n", "")
                JobDescription.append(description)
            description = ''
            break
        else:
            jobTitle = jobTitle.replace("\n", "")
            description=description+''+jobTitle.lower()
    description=description+' '

In [25]:
anotations_dictionarey = list()

In [29]:
''' < Train a New NER > '''
n=0
# Testing the NER
for jobdes in JobDescription:
    annotations = skill_extractor.annotate(jobdes)
    anotations_dictionarey.append(annotations)
    skill_extractor.describe(annotations)
    
    n+=1
    if n==200:
        break

In [30]:
for annotation in anotations_dictionarey:
    annotation = eval(str(annotation))
    pretty_json = json.dumps(annotation, indent=4)
#     print(pretty_json)
    display(skill_extractor.describe(annotation))
    
#     mylist = eval(str(mylist_of_integers))
# json.dumps({'mylist': mylist})